In [806]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import time
from selenium import webdriver
from sklearn.preprocessing import OneHotEncoder

## This script will retrieve key financial indicators for US stocks in any desired industry in the "Stocks by Industry" table from the following website https://www.macrotrends.net/stocks/research. 

## Find all industries with hyperlinks

In [730]:
# url to scrape
url_root = "https://www.macrotrends.net"
url = url_root + "/stocks/research"

# A dictionary of HTTP headers to send to the specified url. Your version of Chrome
headers = {"User-Agent" : "Chrome/97.0.4692.71"}

page = requests.get(url, headers=headers) # establishing connection with web server
page_content = page.content # retrieve content
soup = BeautifulSoup(page_content, 'html.parser') # create BeautifulSoup object
table = soup.find_all("table",{"class":"table table-bordered table-hover"})[0] # find the "Stocks by Industry" table

industryLink = {} # dictionary to store the industry name : link to industry page list of stocks
industryStore = [] # list of each industry

for row in table.find_all("tr")[2:]: # for all data rows
    # find all industry names
    industry = row.find_all("td")[0].find("a").text
    industryStore.append(industry)
    
    # get links to each industry
    link = row.find_all("td")[0].find("a")['href']
    
    # dictionary: add link [0], add randNum for parsing page that link brings you to
    industryLink[industry] = [link, link.split("/")[3]]    
    

## Choose an industry and go to link of the stocks in that industry

### This specific page needed to be accessed using the webdriver from selenium so that it can scrape the data that needs to be loaded to the page.

In [731]:
### choose an industry (names based on industry names on table from url_root)
ind = "Large Cap Pharmaceutical"

### scrape industry table for each stock name
url_ind = industryLink[ind][0]
browser = webdriver.Chrome('/Users/jonathanwozny/Documents/chromedriver') # downloaded webdriver in this location
browser.get(url_root + url_ind) # go to desired url
time.sleep(5) # pause so that page will load
html = browser.page_source # get html
soup = BeautifulSoup(html, 'html.parser') # create BeautifulSoup object
browser.quit()


### Get a list of US-based stocks and their urls in desired industry

In [732]:
table = soup.find_all("div", {"role":"row"})

compNameStore = [] # store the company name
compTickStore = [] # store the company ticker
compLink = {}      # dictionary storing the company ticker : link to company financial page

for row in table:
    # grab the first cell in each 
    first_cell = row.find_all("div", {"role":"gridcell"})[0]
    
    # grab the second cell in each to get the country that it is present
    # only get stocks from the United States
    if row.find_all("div", {"role":"gridcell"})[1].text == "United States": # only retrieve stocks that are based in 
                                                                            # the United States
        # get company name and ticker
        compName = first_cell.text.split("(") # separate company name and ticker
        compName[1] = compName[1].replace(")", "")
        compNameStore.append(compName[0]) # store company name
        compTickStore.append(compName[1]) # store company ticker

        # store in dictionary with tickers as the indices and format link to be more useful for scraping diff. ratios
        link = first_cell.find("a")["href"].split("stock-")[0]
        link = link.replace("_", ".")
        compLink[compName[1]] = link
    

In [733]:
for key in compLink:
    print(key, " ", compLink[key])

JNJ   https://www.macrotrends.net/stocks/charts/JNJ/johnson-johnson/
PFE   https://www.macrotrends.net/stocks/charts/PFE/pfizer/
ABBV   https://www.macrotrends.net/stocks/charts/ABBV/abbvie/
LLY   https://www.macrotrends.net/stocks/charts/LLY/eli-lilly/
MRK   https://www.macrotrends.net/stocks/charts/MRK/merck/
INVA   https://www.macrotrends.net/stocks/charts/INVA/innoviva/


## Get ratios for each company and add to DataFrame

In [909]:
# available pre-calculated ratios
ratios = ["pe-ratio", "price-sales", "price-book", "price-fcf", "current-ratio", "quick-ratio", "debt-equity-ratio",
          "roa", "return-on-tangible-equity"] # "roe"

all_df = pd.DataFrame()      # dataframe for each company to be merged to all_comp_df
all_comp_df = pd.DataFrame() # df storing data from all companies
temp = pd.DataFrame()        # temporary data frame to merge each new ratio into existing dataframe for each company

bool3 = False # in the case that all_comp_df is empty and has nothing to merge

for i, key in enumerate(compLink): # for each company
    bool1 = False # in the case that df is empty
    bool2 = False
    for r in ratios: # for each ratio
        headers = {"User-Agent" : "Chrome/97.0.4692.71"} # A dictionary of HTTP headers to send to the specified url. Your version of Chrome
        page = requests.get(compLink[key] + r, headers=headers) # establishing connection with the web server
        page_content = page.content
        soup = BeautifulSoup(page_content, 'html.parser') # create BeautifulSoup object
        
        table = soup.find_all("table", {"class" : "table"})[0] # find the table for the financial metric

        new_headers = [] # header list for the dataframe

        idx = [1,2,-1] # get date, stock price, and desired metric (will always be last column)
        #new_headers.append([table.find_all("th")[i].text.strip() for i in idx])
        new_headers = [table.find_all("th")[i].text.strip() for i in idx] # get the desired headers

        df = pd.DataFrame(columns = new_headers) # create dataframe with headers --> date, stock price, metric

        idx = [0,1,-1] # get desired indice data (date, stock price) --> different indices when looking at each data entry
        for row in table.find_all("tr")[2:]: # find all data rows
            row_data = [row.find_all("td")[i].text for i in idx] # get row data
            df.loc[len(df)] = row_data # add row data to dataframe
            
        df["Company"] = compTickStore[i] # add the name of the company to the dataframe (can be used as categorical)

        if bool1 == False: # if dataframe has not been merged yet, just set to temp to be merged in next time
            temp = df
            bool1 = True   # don't call this statement again
        elif bool2 == False: # if dataframe has not been merged yet, but temp is made, then merge into all_df
            all_df = pd.merge(temp, df)
            bool2 = True   # don't call this statement anymore
        else:   # merge df into all_df
            all_df = pd.merge(all_df, df)
    
    if bool3 == False: # if all_comp_df is empty
        bool3 = True # don't call this statement again
        all_comp_df = all_df
    else: # if all_comp_df has values, then we can merge all_df
        all_comp_df = all_comp_df.append(all_df, ignore_index = True) # ignore index resets indices of dataframe to make sense
            

## Change data to float from string and into categories

#### Function to change money abbreviation (i.e. B for billions) into float

In [910]:
def value_to_float(x):
    if type(x) == float or type(x) == int: # if it is already a number
        return x
    if '$' in x:
        x = x.replace("$", "") # remove dollar signs
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000 # convert to thousands
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000 # convert to millions
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000 # convert to billions
    if '%' in x:
        return float(x.replace('%', '')) / 100 # remove percentage signs and turn into numerical
    return float(x)

#### Remove unecessary characters (i.e. $, \%, etc.) for each colum and then apply value_to_float to each column

In [911]:
for col in all_comp_df.drop(["Date", "Company"], axis = 1): # convert desired columns to floats
    all_comp_df[col] = all_comp_df[col].apply(value_to_float)

# turn company into categorical
# compTickMapper = dict(map(reversed, enumerate(compTickStore)))
# all_comp_df["Company"] = all_comp_df["Company"].replace(compTickMapper)

# # one hot encode company
# y = OneHotEncoder(sparse=False).fit_transform(all_comp_df.Company)
ohe_df = pd.get_dummies(all_comp_df.Company) # convert Company into categorial (one-hot-encoding)
# ohe_df = pd.DataFrame(y, columns = compTickStore)
all_comp_df_ohe = pd.concat([all_comp_df, ohe_df], axis = 1) # add one-hot-encoded values to dataframe

In [912]:
all_comp_df_ohe

,Date,Stock Price,PE Ratio,Company,Price to Sales Ratio,Price to Book Ratio,Price to FCF Ratio,Current Assets,Current Ratio,Current Assets - Inventory,...,Debt to Equity Ratio,TTM Net Income,Return on Assets,Return on Tangible Equity,ABBV,INVA,JNJ,LLY,MRK,PFE
0,2021-09-30,159.40,23.83,JNJ,4.66,5.97,18.27,5.989000e+10,1.34,4.950000e+10,...,1.55,1.788000e+10,0.1017,-0.9284,0,0,1,0,0,0
1,2021-06-30,161.63,24.31,JNJ,4.84,6.11,18.43,5.377000e+10,1.39,4.367000e+10,...,1.54,1.777000e+10,0.1023,-0.8795,0,0,1,0,0,0
2,2021-03-31,160.26,28.31,JNJ,5.08,6.41,19.69,5.253000e+10,1.28,4.258000e+10,...,1.62,1.512000e+10,0.0894,-0.7316,0,0,1,0,0,0
3,2020-12-31,152.51,27.68,JNJ,4.93,6.34,19.87,5.124000e+10,1.21,4.189000e+10,...,1.76,1.471000e+10,0.0893,-0.7249,0,0,1,0,0,0
4,2020-09-30,143.26,22.53,JNJ,4.74,5.85,20.73,5.758000e+10,1.48,4.798000e+10,...,1.65,1.699000e+10,0.1059,-0.8883,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,2017-12-31,14.19,12.78,INVA,7.85,-5.96,12.00,2.000000e+08,5.77,0.000000e+00,...,-2.51,1.300000e+08,0.3522,-0.4886,0,1,0,0,0,0
240,2017-09-30,14.12,16.23,INVA,8.92,-6.84,14.84,2.200000e+08,6.71,0.000000e+00,...,-2.75,1.000000e+08,0.2634,-0.3347,0,1,0,0,0,0
241,2017-06-30,12.80,16.41,INVA,8.83,-4.72,17.01,2.000000e+08,7.31,0.000000e+00,...,-2.25,9.000000e+07,0.2431,-0.2722,0,1,0,0,0,0
242,2017-03-31,13.83,23.05,INVA,11.29,-4.52,20.98,2.100000e+08,10.47,0.000000e+00,...,-2.17,7.000000e+07,0.1895,-0.2031,0,1,0,0,0,0


## Make Label

### The label column will be the percent (numerical value) of the price growth of the company in the following quarter. I.e. if stock price in q1 of 2019 is 10 and price in q2 of 2019 is 12, then Label for the q1 row will be 0.2

In [913]:
### create label from stock price
all_comp_df_ohe["Label"] = 0 # initialize at 0
drop_ind = [] # indices to drop (no data for the next quarter so cannot have a label)...will be dropped after being used
drop_ind.append(0) # first row will be dropped
for i in all_comp_df_ohe.index: # iterate through every row
    new = float(all_comp_df_ohe["Stock Price"].iloc[i-1]) # new price
    old = float(all_comp_df_ohe["Stock Price"].iloc[i]) # old price
    if (i != 0 or i != len(all_comp_df_ohe)-1): # can't use index -1 or past last index
        if (all_comp_df_ohe.iloc[i-1, all_comp_df_ohe.columns.get_loc("Company")] == all_comp_df_ohe.iloc[i, all_comp_df_ohe.columns.get_loc("Company")]):
            all_comp_df_ohe.iloc[i, all_comp_df_ohe.columns.get_loc("Label")] = (new - old)/old # calculate growth
        else: # if company has changed then this is the most recent quarter for the new company
            drop_ind.append(i)
            
# drop first row for each company (there is no future year to use)
all_comp_df_ohe.drop(index=all_comp_df_ohe.index[[drop_ind]], axis=0, inplace=True)

# drop company series
all_comp_df_ohe.drop(["Stock Price", "Company"], axis=1, inplace = True) # stock price is not needed any more
                                                                         # also drop company after one-hot-encoding performed

/Users/jonathanwozny/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:4307: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


In [915]:
from sklearn.preprocessing import MinMaxScaler

# store copy of all_x
all_normalized = all_comp_df_ohe.copy()

# create list to not scale
non_scaled = ["Date", "Return on Assets", "Return on Tangible Equity","Label"] + compTickStore #, "Return on Equity"

# fit minmax scaler to copy of all_x
scaler = MinMaxScaler(copy=True)
scaler.fit(all_normalized[[col for col in all_comp_df_ohe.drop(non_scaled, axis = 1)]])

# store min-max normalized values
all_normalized[[col for col in all_comp_df_ohe.drop(non_scaled, axis = 1)]] = scaler.transform(all_normalized[[col for col in all_comp_df_ohe.drop(non_scaled, axis = 1)]])


In [916]:
all_normalized

,Date,PE Ratio,Price to Sales Ratio,Price to Book Ratio,Price to FCF Ratio,Current Assets,Current Ratio,Current Assets - Inventory,Quick Ratio,Long Term Debt,...,TTM Net Income,Return on Assets,Return on Tangible Equity,ABBV,INVA,JNJ,LLY,MRK,PFE,Label
1,2021-06-30,0.075410,0.368266,0.516896,0.029380,0.810819,0.003553,0.743193,0.369281,0.776716,...,0.676327,0.1023,-0.8795,0,0,1,0,0,0,-0.013797
2,2021-03-31,0.087818,0.391773,0.520198,0.031951,0.792082,0.002890,0.724643,0.339869,0.775695,...,0.576590,0.0894,-0.7316,0,0,1,0,0,0,0.008549
3,2020-12-31,0.085864,0.377081,0.519428,0.032318,0.772590,0.002469,0.712900,0.323529,0.811438,...,0.561159,0.0893,-0.7249,0,0,1,0,0,0,0.050816
4,2020-09-30,0.069889,0.358472,0.514034,0.034073,0.868389,0.004095,0.816542,0.405229,0.772048,...,0.646970,0.1059,-0.8883,0,0,1,0,0,0,0.064568
5,2020-06-30,0.060955,0.331048,0.511502,0.033032,0.691750,0.002710,0.620660,0.323529,0.693121,...,0.579225,0.0969,-0.7355,0,0,1,0,0,0,0.065685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,2017-12-31,0.039644,0.663075,0.384040,0.016261,0.001360,0.029927,0.000000,0.000000,0.001678,...,0.012420,0.3522,-0.4886,0,1,0,0,0,0,0.174771
240,2017-09-30,0.050346,0.767875,0.374353,0.022055,0.001662,0.035587,0.000000,0.000000,0.001678,...,0.011291,0.2634,-0.3347,0,1,0,0,0,0,0.004958
241,2017-06-30,0.050904,0.759060,0.397688,0.026483,0.001360,0.039200,0.000000,0.000000,0.002115,...,0.010915,0.2431,-0.2722,0,1,0,0,0,0,0.103125
242,2017-03-31,0.071502,1.000000,0.399890,0.034583,0.001511,0.058228,0.000000,0.000000,0.002553,...,0.010162,0.1895,-0.2031,0,1,0,0,0,0,-0.074476


## Write to file, so model algorithm can be written and tested on data

In [ ]:
all_normalized.to_csv("StockData_LargeCapPharma.csv")

### Question...Do I normalize return on assets, return on equity, return on tangible equity (since negative) and already similar to standardization --> should probably normalize for consistency.

## practice with one stock at a time to get structure for larger list of companies scraped from industries table 

In [451]:
ratios = ["pe-ratio", "price-sales", "price-book"]
          
#           , "price-fcf", "current-ratio", "quick-ratio", "debt-equity-ratio",
#          "roe", "roa", "return-on-tangible-equity"]

all_df = pd.DataFrame()
all_new_df = pd.DataFrame()
temp = pd.DataFrame()

bool3 = False
myurl = ["https://www.macrotrends.net/stocks/charts/AAPL/apple/", "https://www.macrotrends.net/stocks/charts/MSFT/microsoft/"]

for i, url in enumerate(myurl):
    bool1 = False
    bool2 = False
    for r in ratios:
        #url = "https://www.macrotrends.net/stocks/charts/AAPL/apple/" + r
        headers = {"User-Agent" : "Chrome/97.0.4692.71"}
        page = requests.get(url + r, headers=headers) # establishing connection with the web server
        page_content = page.content
        soup = BeautifulSoup(page_content, 'html.parser')
        table = soup.find_all("table", {"class" : "table"})[0]

        headers = []
    #     for th in table.find_all("th")[1:]:
    #         headers.append(th.text)
        idx = [1,2,-1]
        headers.append([table.find_all("th")[i].text.strip() for i in idx])

        df = pd.DataFrame(columns = headers)
    #     df.rename(columns={"TTM FCF per Share":"TTM FCF per Share ($)"}, inplace = True)

        idx = [0,1,-1]
        for row in table.find_all("tr")[2:]:
            row_data = [row.find_all("td")[i].text for i in idx]
    #         row_data = [td.text.strip() for td in row.find_all("td")]
            df.loc[len(df)] = row_data
        df["Company"] = i

        if bool1 == False:
            temp = df
            bool1 = True    
        elif bool2 == False:
            all_df = pd.merge(temp, df)
            bool2 = True
        else:   
            all_df = pd.merge(all_df, df)
    #print(all_df)
    
    if bool3 == False:
        #all_new_df.columns = all_df.columns
        bool3 = True
        all_new_df = all_df
    else:
        all_new_df = all_new_df.append(all_df, ignore_index = True)
    print(all_new_df)

        # for col_name in df.columns[1:]:
        #     df[col_name] = df[col_name].apply(lambda x: float(x))



          Date Stock Price PE Ratio Company Price to Sales Ratio  \
0   2021-12-31      177.34    29.36       0                 7.85   
1   2021-09-30      141.11    25.11       0                 6.53   
2   2021-06-30      136.38    26.69       0                 6.70   
3   2021-03-31      121.43    27.26       0                 6.42   
4   2020-12-31      131.71    35.67       0                 7.78   
5   2020-09-30      114.76    35.20       0                 7.37   
6   2020-06-30       90.20    27.44       0                 5.88   
7   2020-03-31       62.71    19.67       0                 4.23   
8   2019-12-31       72.25    22.83       0                 4.95   
9   2019-09-30       54.94    18.54       0                 3.96   
10  2019-06-30       48.36    16.49       0                 3.56   
11  2019-03-31       46.24    15.55       0                 3.46   
12  2018-12-31       38.23    12.58       0                 2.88   
13  2018-09-30       54.52    18.37       0     

In [453]:
all_new_df

,Date,Stock Price,PE Ratio,Company,Price to Sales Ratio,Price to Book Ratio
0,2021-12-31,177.34,29.36,0,7.85,40.29
1,2021-09-30,141.11,25.11,0,6.53,36.74
2,2021-06-30,136.38,26.69,0,6.70,35.13
3,2021-03-31,121.43,27.26,0,6.42,29.29
4,2020-12-31,131.71,35.67,0,7.78,33.46
...,...,...,...,...,...,...
93,2010-12-31,22.04,9.38,1,2.89,3.82
94,2010-09-30,19.22,8.28,1,2.59,3.50
95,2010-06-30,17.96,8.55,1,2.57,3.37
96,2010-03-31,22.76,11.79,1,3.42,4.36
